# Pools

### What we need
- Pool deposits
- Pool withdrawals
- Pool fees
- Pool volume

## Setup

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
from mezo.currency_utils import format_token_columns
from mezo.datetime_utils import format_datetimes
from mezo.currency_config import POOL_TOKEN0_MAP
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import BigQueryClient, SubgraphClient, Web3Client

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [ ]:
POOLS_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/musd-pools-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_POOL_FEES = """
query poolFees($skip: Int!) {
fees(
    first: 1000
    orderBy: timestamp_ 
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    sender
    amount0
    amount1
    contractId_
    transactionHash_
    block_number
    }
}
"""

## Get pool volume data from tigris-pools-mezo

In [ ]:
TIGRIS_POOLS_SUBRAPH = 'https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/tigris-pools-mezo/1.0.0/gn'

GET_POOL_VOLUME = """
query getPoolVolumes($skip: Int!) {
  poolVolumes(
    interval: day
    first: 1000
    orderBy: timestamp_ 
    orderDirection: desc
    skip: $skip
  ) {
  	timestamp
    pool {
      name
      token0 {
        symbol
      }
      token1 {
        symbol
      }
    }
    totalVolume0
    totalVolume1
  }
}
"""

In [ ]:
# Instantiate subgraph
pool_volumes = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_volume_data =  pool_volumes.fetch_subgraph_data(GET_POOL_VOLUME, 'poolVolumes')

In [ ]:
def flatten_json_column(df, json_col, prefix=None):
    """
    Flatten any JSON column into separate columns using pd.json_normalize
    
    Args:
        df: DataFrame with column containing JSON data
        json_col: Name of the column containing JSON data
        prefix: Optional prefix for new column names (defaults to json_col + '_')
    
    Returns:
        DataFrame with flattened JSON data as new columns
    """
    if json_col not in df.columns:
        raise ValueError(f"Column '{json_col}' not found in DataFrame")
    
    # Set default prefix
    if prefix is None:
        prefix = f"{json_col}_"
    
    # Normalize the JSON data
    json_normalized = pd.json_normalize(df[json_col])
    
    # Create the result DataFrame starting with original data
    result_df = df.copy()
    
    # Add all flattened columns with prefix
    for col in json_normalized.columns:
        # Replace dots with underscores for cleaner column names
        clean_col_name = col.replace('.', '_')
        new_col_name = f"{prefix}{clean_col_name}"
        result_df[new_col_name] = json_normalized[col]
    
    return result_df

In [ ]:
# Save pool volume data df

volume_df = pd.DataFrame(pool_volume_data)
display(volume_df)

# flatten col
volume_df_flattened = flatten_json_column(volume_df, 'pool')
volume_df_flattened = volume_df_flattened.drop(columns=['pool'])

In [ ]:
volume_df_flattened.head()

# upload to bigquery

In [ ]:
volume_df_flattened['pool_token0_symbol'] = volume_df_flattened['pool_token0_symbol'].replace(
    {
        'mUSDC': 'USDC', 
        'mUSDT':'USDT'
    }
)

volume_df_flattened['pool_token1_symbol'] = volume_df_flattened['pool_token1_symbol'].replace(
    {
        'mUSDC': 'USDC', 
        'mUSDT':'USDT'
    }
)

In [49]:
# ADD TO CURRENCY_CONFIG

TIGRIS_MAP = {
    'Volatile AMM - BTC/MUSD': 'musd_btc_pool',
    'Stable AMM - mUSDC/MUSD': 'musd_musdc_pool',
    'Stable AMM - MUSD/mUSDT': 'musd_musdt_pool',
    'Stable AMM - mUSDC/mUSDT': 'musdc_musdt_pool'
}

In [ ]:
volume_df_flattened['pool'] = volume_df_flattened['pool_name'].map(TIGRIS_MAP)

In [ ]:
vol_df_test = format_datetimes(volume_df_flattened, ['timestamp'])
vol_df_test.head()

In [ ]:
vol_df_test_2 = format_token_columns(vol_df_test, ['totalVolume0'], 'pool_token0_symbol')
vol_df_test_2.head()

In [ ]:
vol_df_test_3 = format_token_columns(vol_df_test_2, ['totalVolume1'], 'pool_token1_symbol')

In [ ]:
from mezo.currency_utils import get_token_prices
from mezo.currency_config import TOKENS_ID_MAP

def add_usd_conversions(df, token_column, tokens_id_map, amount_columns=None):
    """
    Add USD price conversions to any token data
    
    Args:
        df: DataFrame containing token data
        token_column: Name of column containing token identifiers
        tokens_id_map: Dictionary mapping tokens to CoinGecko IDs
        amount_columns: List of amount columns to convert, or None for auto-detection
    
    Returns:
        DataFrame with USD conversion columns added
    """
    if token_column not in df.columns:
        raise ValueError(f"Column '{token_column}' not found in DataFrame")
    
    # Get token prices
    prices = get_token_prices()
    if prices is None or prices.empty:
        raise ValueError("No token prices received from API")
    
    token_usd_prices = prices.T.reset_index()
    df_result = df.copy()
    df_result['index'] = df_result[token_column].map(tokens_id_map)
    
    df_with_usd = pd.merge(df_result, token_usd_prices, how='left', on='index')
    
    # Set MUSD price to 1.0 (1:1 with USD)
    df_with_usd.loc[df_with_usd[token_column] == 'MUSD', 'usd'] = 1.0
    
    # Auto-detect amount columns if not provided
    if amount_columns is None:
        amount_columns = [col for col in df.columns if 'amount' in col.lower() and col != 'amount_usd']
    
    # Add USD conversion for each amount column
    for col in amount_columns:
        if col in df_with_usd.columns:
            usd_col_name = f"{col}_usd" if not col.endswith('_usd') else col
            df_with_usd[usd_col_name] = df_with_usd[col] * df_with_usd['usd']
    
    return df_with_usd

In [ ]:
vol_df_test_4 = add_usd_conversions(vol_df_test_3, 'pool_token0_symbol', TOKENS_ID_MAP, 'totalVolume0')
vol_df_test_4.head()

In [ ]:
vol_df_test_4['count'] = 1

## Get pool deposits and withdrawals

### Deposits

In [ ]:
# Fetch pool fees data
pool_fees = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_fees_data = pool_fees.fetch_subgraph_data(GET_TOTAL_POOL_FEES, 'feesStats_collection')

# Save pool fees data df
fees_df = pd.DataFrame(pool_fees_data)
display(fees_df)

# Flatten JSON column
fees_df_flat = flatten_json_column(fees_df, 'pool')
fees_df_flat = fees_df_flat.drop(columns=['pool'])

# Replace mUSDC and mUSDT with USDC and USDT
fees_df_flat['pool_token0_symbol'] = fees_df_flat['pool_token0_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

fees_df_flat['pool_token1_symbol'] = fees_df_flat['pool_token1_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

# Format timestamps
fees_df_formatted = format_datetimes(fees_df_flat, ['timestamp'])

# Format token columns for totalFees0 and totalFees1
fees_df_token0 = format_token_columns(fees_df_formatted, ['totalFees0'], 'pool_token0_symbol')
fees_df_token1 = format_token_columns(fees_df_token0, ['totalFees1'], 'pool_token1_symbol')

# Add USD conversions for totalFees0
fees_df_usd0 = add_usd_conversions(fees_df_token1, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalFees0'])

# Add USD conversions for totalFees1 
fees_df_usd1 = add_usd_conversions(fees_df_usd0, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalFees1'])

# Add count column
fees_df_usd1['count'] = 1

print("Processed fees data:")
display(fees_df_usd1.head())
print(f"\nTotal fee records: {len(fees_df_usd1)}")
print(f"Date range: {fees_df_usd1['timestamp'].min()} to {fees_df_usd1['timestamp'].max()}")

fees_df_final = fees_df_usd1

In [ ]:
GET_DEPOSITS = """
    query getMints ($skip: Int!) {
    mints(
        first: 1000
        orderBy: timestamp_
        orderDirection: desc
        skip: $skip
    ) {
        timestamp_
        sender
        amount0
        amount1
        contractId_
        transactionHash_
    }
}
"""

In [ ]:
# Instantiate subgraph
pool_deposits = SubgraphClient(
    url = POOLS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_deposit_data =  pool_deposits.fetch_subgraph_data(GET_DEPOSITS, 'mints')

In [ ]:
raw = pd.DataFrame(pool_deposit_data)
display(raw)

ddf = raw.copy()

In [ ]:
from mezo.currency_config import POOL_TOKEN_PAIRS, TOKENS_ID_MAP, POOLS_MAP
from mezo.currency_utils import format_pool_token_columns, add_pool_usd_conversions

ddf['pool'] = ddf['contractId_'].map(POOLS_MAP)
ddf = format_datetimes(ddf, ['timestamp_'])
ddf = format_pool_token_columns(ddf, 'contractId_', POOL_TOKEN_PAIRS)
ddf = add_pool_usd_conversions(ddf, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)

ddf

### Withdrawals

In [ ]:
GET_WITHDRAWALS = """
query getWithdrawals($skip: Int!) {
  burns(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    sender
    to
    amount0
    amount1
    contractId_
    transactionHash_
  }
}
"""

In [ ]:
# Instantiate subgraph
pool_withdrawals = SubgraphClient(
    url = POOLS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_wit_data =  pool_withdrawals.fetch_subgraph_data(GET_WITHDRAWALS, 'burns')

In [ ]:
# Process withdrawals data in the same way as deposits
withdrawals_df = pd.DataFrame(pool_wit_data)
wdf = withdrawals_df.copy()

# Apply same processing steps as deposits
wdf['pool'] = wdf['contractId_'].map(POOLS_MAP)
wdf = format_datetimes(wdf, ['timestamp_'])
wdf = format_pool_token_columns(wdf, 'contractId_', POOL_TOKEN_PAIRS)
wdf = add_pool_usd_conversions(wdf, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)

print("Processed withdrawals data:")
display(wdf.head())
print(f"\nTotal withdrawals: {len(wdf)}")
print(f"Date range: {wdf['timestamp_'].min()} to {wdf['timestamp_'].max()}")

wdf

## Get pool fees

In [ ]:
GET_TOTAL_POOL_FEES = """
query getFeeStats($skip: Int!) {
  feesStats_collection(
    interval: day
    first: 1000
    orderBy: timestamp
    orderDirection: desc
    skip: $skip
  ) {
    timestamp
    pool {
      name
      token0 {
        symbol
      }
      token1 {
        symbol
      }
    }
    totalFees0
    totalFees1
  }
}
"""

In [ ]:
# Instantiate subgraph
fees_raw = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
fees_data =  fees_raw.fetch_subgraph_data(GET_TOTAL_POOL_FEES, 'feesStats_collection')

In [43]:
# Save pool volume data df

fees_df = pd.DataFrame(fees_data)

# flatten col
fees_df_flat = flatten_json_column(fees_df, 'pool')
fees_df_flat = fees_df_flat.drop(columns=['pool'])

fees_df_flat.head(25)

,timestamp,totalFees0,totalFees1,pool_name,pool_token0_symbol,pool_token1_symbol
0,1753920000000000,394614,353969857503108807,Stable AMM - mUSDC/MUSD,mUSDC,MUSD
1,1753920000000000,275892697088849,32722649328737625868,Volatile AMM - BTC/MUSD,BTC,MUSD
2,1753920000000000,68676060200000000,29658,Stable AMM - MUSD/mUSDT,MUSD,mUSDT
3,1753920000000000,27000,29175,Stable AMM - mUSDC/mUSDT,mUSDC,mUSDT
4,1753833600000000,78516530092120,9569684839225190733,Volatile AMM - BTC/MUSD,BTC,MUSD
5,1753833600000000,162327,386213572416192596,Stable AMM - mUSDC/MUSD,mUSDC,MUSD
6,1753833600000000,210391560000000000,131357,Stable AMM - MUSD/mUSDT,MUSD,mUSDT
7,1753833600000000,128927,56750,Stable AMM - mUSDC/mUSDT,mUSDC,mUSDT
8,1753747200000000,126262768850600,14473349464047000000,Volatile AMM - BTC/MUSD,BTC,MUSD
9,1753747200000000,610087,561213737739257179,Stable AMM - mUSDC/MUSD,mUSDC,MUSD


In [44]:
# Replace mUSDC and mUSDT with USDC and USDT
fees_df_flat['pool_token0_symbol'] = fees_df_flat['pool_token0_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

fees_df_flat['pool_token1_symbol'] = fees_df_flat['pool_token1_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

In [45]:
# Format timestamps
fees_df_flat = format_datetimes(fees_df_flat, ['timestamp'])

fees_df_flat

,timestamp,totalFees0,totalFees1,pool_name,pool_token0_symbol,pool_token1_symbol
0,2025-07-31,394614,353969857503108807,Stable AMM - mUSDC/MUSD,USDC,MUSD
1,2025-07-31,275892697088849,32722649328737625868,Volatile AMM - BTC/MUSD,BTC,MUSD
2,2025-07-31,68676060200000000,29658,Stable AMM - MUSD/mUSDT,MUSD,USDT
3,2025-07-31,27000,29175,Stable AMM - mUSDC/mUSDT,USDC,USDT
4,2025-07-30,78516530092120,9569684839225190733,Volatile AMM - BTC/MUSD,BTC,MUSD
...,...,...,...,...,...,...
67,2025-07-08,750000000000,0,Volatile AMM - BTC/MUSD,BTC,MUSD
68,2025-07-07,0,56730000000000000,Volatile AMM - BTC/MUSD,BTC,MUSD
69,2025-07-07,402,0,Stable AMM - mUSDC/MUSD,USDC,MUSD
70,2025-06-26,0,100,Stable AMM - MUSD/mUSDT,MUSD,USDT


In [46]:
# Format token columns for totalFees0 and totalFees1
fees_df_flat = format_token_columns(fees_df_flat, ['totalFees0'], 'pool_token0_symbol')
fees_df_flat = format_token_columns(fees_df_flat, ['totalFees1'], 'pool_token1_symbol')

fees_df_flat

,timestamp,totalFees0,totalFees1,pool_name,pool_token0_symbol,pool_token1_symbol
0,2025-07-31,3.946140e-01,0.353970,Stable AMM - mUSDC/MUSD,USDC,MUSD
1,2025-07-31,2.758927e-04,32.722649,Volatile AMM - BTC/MUSD,BTC,MUSD
2,2025-07-31,6.867606e-02,0.029658,Stable AMM - MUSD/mUSDT,MUSD,USDT
3,2025-07-31,2.700000e-02,0.029175,Stable AMM - mUSDC/mUSDT,USDC,USDT
4,2025-07-30,7.851653e-05,9.569685,Volatile AMM - BTC/MUSD,BTC,MUSD
...,...,...,...,...,...,...
67,2025-07-08,7.500000e-07,0.000000,Volatile AMM - BTC/MUSD,BTC,MUSD
68,2025-07-07,0.000000e+00,0.056730,Volatile AMM - BTC/MUSD,BTC,MUSD
69,2025-07-07,4.020000e-04,0.000000,Stable AMM - mUSDC/MUSD,USDC,MUSD
70,2025-06-26,0.000000e+00,0.000100,Stable AMM - MUSD/mUSDT,MUSD,USDT


In [47]:
# Add USD conversions for totalFees0
fees_df_flat = add_usd_conversions(fees_df_flat, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalFees0'])

# Add USD conversions for totalFees1 
fees_df_flat = add_usd_conversions(fees_df_flat, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalFees1'])

# Add count column
fees_df_flat['count'] = 1

print("Processed fees data:")
display(fees_df_flat.head())
print(f"\nTotal fee records: {len(fees_df_flat)}")
print(f"Date range: {fees_df_flat['timestamp'].min()} to {fees_df_flat['timestamp'].max()}")

fees_df_final = fees_df_flat

Processed fees data:


,timestamp,totalFees0,totalFees1,pool_name,pool_token0_symbol,pool_token1_symbol,index,usd_x,totalFees0_usd,usd_y,usd,totalFees1_usd,count
0,2025-07-31,0.394614,0.353970,Stable AMM - mUSDC/MUSD,USDC,MUSD,NaN,0.999704,0.394497,NaN,1.0,0.353970,1
1,2025-07-31,0.000276,32.722649,Volatile AMM - BTC/MUSD,BTC,MUSD,NaN,113170.000000,31.222777,NaN,1.0,32.722649,1
2,2025-07-31,0.068676,0.029658,Stable AMM - MUSD/mUSDT,MUSD,USDT,tether,1.000000,0.068676,0.999463,NaN,NaN,1
3,2025-07-31,0.027000,0.029175,Stable AMM - mUSDC/mUSDT,USDC,USDT,tether,0.999704,0.026992,0.999463,NaN,NaN,1
4,2025-07-30,0.000079,9.569685,Volatile AMM - BTC/MUSD,BTC,MUSD,NaN,113170.000000,8.885716,NaN,1.0,9.569685,1



Total fee records: 72
Date range: 2025-05-28 to 2025-07-31


In [51]:
fees_df_final['pool'] = fees_df_final['pool_name'].map(TIGRIS_MAP)